# 1. Installation

In [ ]:
! pip install snap

In [ ]:
! pip install cvxpy

####  The original source code of snapvx.py is out of date, without renewal these days. So we've written a copy which is suitable to run our project code now.

In [ ]:
! python setup.py install --user

# 2. Import

In [ ]:
import numpy as np
from cvxpy import *
from snapvx import *
import time

# 3. Implementation

## 3.1 Random Network

In [ ]:
def RandomGraph(num_nodes, num_edges):
    
    print('RandomGraph: {} nodes, {} edges'.format(num_nodes, num_edges))
    
    np.random.seed(1)
    
    # Generate graph
    
    snapGraph = GenRndGnm(PUNGraph, num_nodes, num_edges)

    while snapGraph.CntDegNodes(0) != 0:
        snapGraph = GenRndGnm(PUNGraph, num_nodes, num_edges)
    
    # Save edge
    
    snap.SaveEdgeList(snapGraph, './RandomGraph/RandomGraph_{}_{}.xlsx'.format(num_nodes, num_edges), "List of edges")

    return snapGraph

In [ ]:
def RandomGraph_Solve(snapGraph, num_nodes, num_edges, maxiters=250):
    gvx=TGraphVX(snapGraph)
    
    # Define edge weights
    c = dict() # edge weights
    d_list =[0] * num_nodes # node degrees
    
    for ei in gvx.Edges():
    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId()) #  GetSrcNId: Source Node / Node with smaller ID,  GetDstNId()): Destination Node / Node with greater ID 
        weight = np.random.uniform(0.5,1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.
        c[(src_id, dst_id)] = weight # key: Edge(Node_small, Node_great), value: weight
        (d_list[src_id],d_list[dst_id]) = (d_list[src_id] + weight, d_list[dst_id] + weight)
   
    f1 = open('./RandomGraph/RandomGraph_{}_{}_weighted.csv'.format(num_nodes, num_edges), 'w+')

    with open('./RandomGraph/RandomGraph_{}_{}.xlsx'.format(num_nodes, num_edges), 'r') as f2:
        for l in f2.readlines():
            if l.startswith('#'):
                if l.startswith('# NodeId'):
                    f1.write('Target, Source, Weight\n') 
                continue
        
            q = l.split('\t')
            q[1] = q[1].strip('\n')

            f1.write(str(q[0])+','+str(q[1])+','+str(c[(int(q[0]), int(q[1]))])+'\n')


    f2.close()
    f1.close()
    
    print('Edge file ready.')
    
    # Define parameters to solve pagerank problem

    beta=Parameter(value = 0.85) # transition probability

    alpha = 1/beta.value -1
    
    v=Parameter(num_nodes) # random teleportation distribution vector
    
    val=np.random.uniform(size=num_nodes) 
    # val=np.ones(num_nodes) # Use np.ones if all equally likely (low=0, high=1)

    # print(val)
    v.value = val/val.sum()
    
    #------------ Solve Using SnapVX (approach based on: http://jmlr.org/proceedings/papers/v32/gleich14.pdf) --------------
    # Set node objectives
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        x = Variable(1,name='x')
        d=d_list[n_id] 
        gvx.SetNodeObjective(n_id,alpha*v.value[n_id]*square(1-x) + alpha*(d-v.value[n_id])*square(0-x)) # x_s = 1, x_t = 0

    # Set edge objectives
    for ei in gvx.Edges():    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId())     
        (src_vars, dst_vars) = (gvx.GetNodeVariables(src_id), gvx.GetNodeVariables(dst_id))
        edge_obj=  c[(src_id, dst_id)]*square(src_vars['x']-dst_vars['x'])
        gvx.SetEdgeObjective(src_id,dst_id,edge_obj)
        
    # UseADMM=True  
    
    start = time.time()

    gvx.Solve(UseADMM=True, EpsAbs=0.0002, EpsRel=0.0002, MaxIters=maxiters) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_True = end - start # runtime
    
    loss_True = float(gvx.value)
    
    print("UseADMM=True, Runtime: {}s, loss: {}".format(runtime_True, loss_True)) 
    
    solution_nodes_True = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x'))) 
            solution_nodes_True.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_True)
    
    solution_True = (sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
#     print(sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
    
    # UseADMM=False
    
    start = time.time()

    gvx.Solve(UseADMM=False) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_False = end - start # runtime
    
    loss_False = float(gvx.value)
    
    print("UseADMM=False, Runtime: {}s, loss: {}".format(runtime_False, loss_False))
    
    solution_nodes_False = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
            solution_nodes_False.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_False)

    solution_False = (sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))

#     print(sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))
                    
    return runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False


In [ ]:
node_lst = []
edge_lst = []

runtime_lst_True = []
runtime_lst_False = []

loss_lst_True = []
loss_lst_False = []

solutions_True = []
solutions_False = []

for n, e in [(20, 24), (50, 60), (100, 120)]:
    
    RandomGraph_n = RandomGraph(int(n), int(e))
    runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False = RandomGraph_Solve(RandomGraph_n, n, e)
    
    node_lst.append(n)
    edge_lst.append(n)
    
    runtime_lst_True.append(runtime_True)
    loss_lst_True.append(loss_True)
    solutions_True.append(solution_True)
    
    runtime_lst_False.append(runtime_False)
    loss_lst_False.append(loss_False)
    solutions_False.append(solution_False)

In [ ]:
with open('./RandomGraph/RandomGraph_Result.csv', 'w') as f:  
    f.write('nodes, edges, runtime_True, runtime_False, loss_True, loss_False, solution_possibility_True, solution_node_True, solution_possibility_False, solution_node_False\n')
    for n in range(len(node_lst)):
        n = int(n)
        f.write(str(node_lst[n])+','+str(edge_lst[n])+','+str(runtime_lst_True[n])+','+str(runtime_lst_False[n])+','+str(loss_lst_True[n])+','+str(loss_lst_False[n])+','+str(solutions_True[n][0])+','+str(solutions_True[n][1])+','+str(solutions_False[n][0])+','+str(solutions_False[n][1])+'\n')
f.close()

## 3.2 Complete Network

In [ ]:
def CompleteGraph(num_nodes):
      
    print('CompleteGraph: {} nodes'.format(num_nodes))
    
    np.random.seed(1)
    
    # Generate graph
    
    snapGraph = GenFull(PUNGraph, num_nodes)

    while snapGraph.CntDegNodes(0) != 0:
        snapGraph = GenFull(PUNGraph, num_nodes)
    
    # Save edge
    
    snap.SaveEdgeList(snapGraph, './CompleteGraph/CompleteGraph_{}.xlsx'.format(num_nodes), "List of edges")
    
    return snapGraph


In [ ]:
def CompleteGraph_Solve(snapGraph, num_nodes, maxiters=250):
    gvx=TGraphVX(snapGraph)
    
    # Define edge weights
    c = dict() # edge weights
    d_list =[0] * num_nodes # node degrees
    
    for ei in gvx.Edges():
    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId()) #  GetSrcNId: Source Node / Node with smaller ID,  GetDstNId()): Destination Node / Node with greater ID 
        weight = np.random.uniform(0.5,1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.
    #     weight = np.random.uniform(0, 1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.

        c[(src_id, dst_id)] = weight # key: Edge(Node_small, Node_great), value: weight
        (d_list[src_id],d_list[dst_id]) = (d_list[src_id] + weight, d_list[dst_id] + weight)

    
    f1 = open('./CompleteGraph/CompleteGraph_{}_weighted.csv'.format(num_nodes), 'w+')


    with open('./CompleteGraph/CompleteGraph_{}.xlsx'.format(num_nodes), 'r') as f2:
        for l in f2.readlines():
            if l.startswith('#'):
                if l.startswith('# NodeId'):
                    f1.write('Target, Source, Weight\n') 
                continue
        
            q = l.split('\t')
            q[1] = q[1].strip('\n')

            f1.write(str(q[0])+','+str(q[1])+','+str(c[(int(q[0]), int(q[1]))])+'\n')

    f2.close()
    f1.close()
    
    print('Edge file ready.')

    
    # Define parameters to solve pagerank problem

    beta=Parameter(value = 0.85) # transition probability

    alpha = 1/beta.value -1
    
    v=Parameter(num_nodes) # random teleportation distribution vector
    
    val=np.random.uniform(size=num_nodes) 
    # val=np.ones(num_nodes) # Use np.ones if all equally likely (low=0, high=1)

    # print(val)
    v.value = val/val.sum()
    
    #------------ Solve Using SnapVX (approach based on: http://jmlr.org/proceedings/papers/v32/gleich14.pdf) --------------
    # Set node objectives
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        x = Variable(1,name='x')
        d=d_list[n_id] 
        gvx.SetNodeObjective(n_id,alpha*v.value[n_id]*square(1-x) + alpha*(d-v.value[n_id])*square(0-x)) # x_s = 1, x_t = 0

    # Set edge objectives
    for ei in gvx.Edges():    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId())     
        (src_vars, dst_vars) = (gvx.GetNodeVariables(src_id), gvx.GetNodeVariables(dst_id))
        edge_obj=  c[(src_id, dst_id)]*square(src_vars['x']-dst_vars['x'])
        gvx.SetEdgeObjective(src_id,dst_id,edge_obj)
        
    # UseADMM=True  
    
    start = time.time()

    gvx.Solve(UseADMM=True, EpsAbs=0.0002, EpsRel=0.0002, MaxIters=maxiters) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_True = end - start # runtime
    
    loss_True = float(gvx.value)
    
    print("UseADMM=True, Runtime: {}s, loss: {}".format(runtime_True, loss_True)) 
    
    solution_nodes_True = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x'))) 
            solution_nodes_True.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_True)
    
    solution_True = (sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
#     print(sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
    
    # UseADMM=False
    
    start = time.time()

    gvx.Solve(UseADMM=False) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_False = end - start # runtime
    
    loss_False = float(gvx.value)
    
    print("UseADMM=False, Runtime: {}s, loss: {}".format(runtime_False, loss_False))
    
    solution_nodes_False = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
            solution_nodes_False.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_False)

    solution_False = (sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))

#     print(sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))
                
    
    return runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False


In [ ]:
node_lst = []
edge_lst = []

runtime_lst_True = []
runtime_lst_False = []

loss_lst_True = []
loss_lst_False = []

solutions_True = []
solutions_False = []

for n in np.arange(10, 21, 2):
    
    CompleteGraph_n = CompleteGraph(int(n))
    runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False = CompleteGraph_Solve(CompleteGraph_n, int(n))
    
    node_lst.append(n)
    edge_lst.append(n)
    
    runtime_lst_True.append(runtime_True)
    loss_lst_True.append(loss_True)
    solutions_True.append(solution_True)
    
    runtime_lst_False.append(runtime_False)
    loss_lst_False.append(loss_False)
    solutions_False.append(solution_False)


In [ ]:
with open('./CompleteGraph/CompleteGraph_Result.csv', 'w') as f:  
    f.write('nodes, edges, runtime_True, runtime_False, loss_True, loss_False, solution_possibility_True, solution_node_True, solution_possibility_False, solution_node_False\n')
    for n in range(len(node_lst)):
        n = int(n)
        f.write(str(node_lst[n])+','+str(edge_lst[n])+','+str(runtime_lst_True[n])+','+str(runtime_lst_False[n])+','+str(loss_lst_True[n])+','+str(loss_lst_False[n])+','+str(solutions_True[n][0])+','+str(solutions_True[n][1])+','+str(solutions_False[n][0])+','+str(solutions_False[n][1])+'\n')
f.close()

## 3.3 Circular Network

In [ ]:
def CircularGraph(num_nodes):
    
    print('CircularGraph: {} nodes'.format(num_nodes))
    
    np.random.seed(1)
    
    # Generate graph
    
    snapGraph = GenCircle(PUNGraph, num_nodes)

    while snapGraph.CntDegNodes(0) != 0:
        snapGraph = GenCircle(PUNGraph, num_nodes)
    
    # Save edge
    
    snap.SaveEdgeList(snapGraph, './CircularGraph/CircularGraph_{}.xlsx'.format(num_nodes), "List of edges")

    return snapGraph

In [ ]:
def CircularGraph_Solve(snapGraph, num_nodes, maxiters=250):
    gvx=TGraphVX(snapGraph)
    
    # Define edge weights
    c = dict() # edge weights
    d_list =[0] * num_nodes # node degrees
    
    for ei in gvx.Edges():
    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId()) #  GetSrcNId: Source Node / Node with smaller ID,  GetDstNId()): Destination Node / Node with greater ID 
        weight = np.random.uniform(0.5,1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.
    #     weight = np.random.uniform(0, 1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.

        c[(src_id, dst_id)] = weight # key: Edge(Node_small, Node_great), value: weight
        (d_list[src_id],d_list[dst_id]) = (d_list[src_id] + weight, d_list[dst_id] + weight)

    
    f1 = open('./CircularGraph/CircularGraph_{}_weighted.csv'.format(num_nodes), 'w+')


    with open('./CircularGraph/CircularGraph_{}.xlsx'.format(num_nodes), 'r') as f2:
        for l in f2.readlines():
            if l.startswith('#'):
                if l.startswith('# NodeId'):
                    f1.write('Target, Source, Weight\n') 
                continue
        
            q = l.split('\t')
#             print(q)
            q[1] = q[1].strip('\n')

            f1.write(str(q[0])+','+str(q[1])+','+str(c[(int(q[0]), int(q[1]))])+'\n')
#             print(q)

    f2.close()

    f1.close()
    
    print('Edge file ready.')

    
    # Define parameters to solve pagerank problem

    beta=Parameter(value = 0.85) # transition probability

    alpha = 1/beta.value -1
    
    v=Parameter(num_nodes) # random teleportation distribution vector
    
    val=np.random.uniform(size=num_nodes) 
    # val=np.ones(num_nodes) # Use np.ones if all equally likely (low=0, high=1)

    # print(val)
    v.value = val/val.sum()
    
    #------------ Solve Using SnapVX (approach based on: http://jmlr.org/proceedings/papers/v32/gleich14.pdf) --------------
    # Set node objectives
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        x = Variable(1,name='x')
        d=d_list[n_id] 
        gvx.SetNodeObjective(n_id,alpha*v.value[n_id]*square(1-x) + alpha*(d-v.value[n_id])*square(0-x)) # x_s = 1, x_t = 0

    # Set edge objectives
    for ei in gvx.Edges():    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId())     
        (src_vars, dst_vars) = (gvx.GetNodeVariables(src_id), gvx.GetNodeVariables(dst_id))
        edge_obj=  c[(src_id, dst_id)]*square(src_vars['x']-dst_vars['x'])
        gvx.SetEdgeObjective(src_id,dst_id,edge_obj)
        
    # UseADMM=True  
    
    start = time.time()

    gvx.Solve(UseADMM=True, EpsAbs=0.0002, EpsRel=0.0002, MaxIters=maxiters) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_True = end - start # runtime
    
    loss_True = float(gvx.value)
    
    print("UseADMM=True, Runtime: {}s, loss: {}".format(runtime_True, loss_True)) 
    
    solution_nodes_True = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
            solution_nodes_True.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_True)
    
    solution_True = (sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
#     print(sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
                        
    # UseADMM=False
    
    start = time.time()

    gvx.Solve(UseADMM=False, EpsAbs=0.0002, EpsRel=0.0002) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_False = end - start # runtime
    
    loss_False = float(gvx.value)
    
    print("UseADMM=False, Runtime: {}s, loss: {}".format(runtime_False, loss_False))
    
    solution_nodes_False = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x'))) 
            solution_nodes_False.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_False)

    solution_False = (sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))

#     print(sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))
                
    
    return runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False



In [ ]:
node_lst = []
edge_lst = []

runtime_lst_True = []
runtime_lst_False = []

loss_lst_True = []
loss_lst_False = []

solutions_True = []
solutions_False = []

for n in [20, 50, 100, 500, 1000]:
    
    CircularGraph_n = CircularGraph(int(n))
    runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False = CircularGraph_Solve(CircularGraph_n, int(n))
    
    node_lst.append(n)
    edge_lst.append(n)
    
    runtime_lst_True.append(runtime_True)
    loss_lst_True.append(loss_True)
    solutions_True.append(solution_True)
    
    runtime_lst_False.append(runtime_False)
    loss_lst_False.append(loss_False)
    solutions_False.append(solution_False)

In [ ]:
with open('./CircularGraph/CircularGraph_Result.csv', 'w') as f:  
    f.write('nodes, edges, runtime_True, runtime_False, loss_True, loss_False, solution_possibility_True, solution_node_True, solution_possibility_False, solution_node_False\n')
    for n in range(len(node_lst)):
        n = int(n)
        f.write(str(node_lst[n])+','+str(edge_lst[n])+','+str(runtime_lst_True[n])+','+str(runtime_lst_False[n])+','+str(loss_lst_True[n])+','+str(loss_lst_False[n])+','+str(solutions_True[n][0])+','+str(solutions_True[n][1])+','+str(solutions_False[n][0])+','+str(solutions_False[n][1])+'\n')
f.close()

## 3.4 Star Network

In [ ]:
def StarGraph(num_nodes):
    
    print('StarGraph: {} nodes'.format(num_nodes))
    
    np.random.seed(1)
    
    # Generate graph
    
    snapGraph = GenStar(PUNGraph, num_nodes, False)

    while snapGraph.CntDegNodes(0) != 0:
        snapGraph = GenStar(PUNGraph, num_nodes, False)
    
    # Save edge
    
    snap.SaveEdgeList(snapGraph, './StarGraph/StarGraph_{}.xlsx'.format(num_nodes), "List of edges")

    return snapGraph

In [ ]:
def StarGraph_Solve(snapGraph, num_nodes, maxiters=250):
    
    gvx=TGraphVX(snapGraph)
    
    # Define edge weights
    c = dict() # edge weights
    d_list =[0] * num_nodes # node degrees
    
    for ei in gvx.Edges():
    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId()) #  GetSrcNId: Source Node / Node with smaller ID,  GetDstNId()): Destination Node / Node with greater ID 
        weight = np.random.uniform(0.5,1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.
    #     weight = np.random.uniform(0, 1) # Draw random samples from a uniform distribution [low=0.5, high=1), and high=1 is excluded.

        c[(src_id, dst_id)] = weight # key: Edge(Node_small, Node_great), value: weight
        (d_list[src_id],d_list[dst_id]) = (d_list[src_id] + weight, d_list[dst_id] + weight)

    
    f1 = open('./StarGraph/StarGraph_{}_weighted.csv'.format(num_nodes), 'w+')


    with open('./StarGraph/StarGraph_{}.xlsx'.format(num_nodes), 'r') as f2:
        for l in f2.readlines():
            if l.startswith('#'):
                if l.startswith('# NodeId'):
                    f1.write('Target, Source, Weight\n') 
                continue
        
            q = l.split('\t')
#             print(q)
            q[1] = q[1].strip('\n')

            f1.write(str(q[0])+','+str(q[1])+','+str(c[(int(q[0]), int(q[1]))])+'\n')
#             print(q)

    f2.close()

    f1.close()
    
    print('Edge file ready.')


    
    # Define parameters to solve pagerank problem

    beta=Parameter(value = 0.85) # transition probability

    alpha = 1/beta.value -1
    
    v=Parameter(num_nodes) # random teleportation distribution vector
    
    val=np.random.uniform(size=num_nodes) 
    # val=np.ones(num_nodes) # Use np.ones if all equally likely (low=0, high=1)

    # print(val)
    v.value = val/val.sum()
    
    #------------ Solve Using SnapVX (approach based on: http://jmlr.org/proceedings/papers/v32/gleich14.pdf) --------------
    # Set node objectives
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        x = Variable(1,name='x')
        d=d_list[n_id] 
        gvx.SetNodeObjective(n_id,alpha*v.value[n_id]*square(1-x) + alpha*(d-v.value[n_id])*square(0-x)) # x_s = 1, x_t = 0

    # Set edge objectives
    for ei in gvx.Edges():    
        (src_id, dst_id) = (ei.GetSrcNId(), ei.GetDstNId())     
        (src_vars, dst_vars) = (gvx.GetNodeVariables(src_id), gvx.GetNodeVariables(dst_id))
        edge_obj=  c[(src_id, dst_id)]*square(src_vars['x']-dst_vars['x'])
        gvx.SetEdgeObjective(src_id,dst_id,edge_obj)
        
    # UseADMM=True  
    
    start = time.time()

    gvx.Solve(UseADMM=True, EpsAbs=0.0002, EpsRel=0.0002, MaxIters=maxiters) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_True = end - start # runtime
    
    loss_True = float(gvx.value)
    
    print("UseADMM=True, Runtime: {}s, loss: {}".format(runtime_True, loss_True)) 
    
    solution_nodes_True = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
            solution_nodes_True.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_True)
    
    solution_True = (sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
#     print(sorted(solution_nodes_True)[-1], solution_nodes_True.index(sorted(solution_nodes_True)[-1]))
                        
    # UseADMM=False
    
    start = time.time()

    gvx.Solve(UseADMM=False, EpsAbs=0.0002, EpsRel=0.0002) # If UseADMM=True, set EpsAbs and EpsRel to 0.0002 for better convergence

    end = time.time()
    runtime_False = end - start # runtime
    
    loss_False = float(gvx.value)
    
    print("UseADMM=False, Runtime: {}s, loss: {}".format(runtime_False, loss_False))
    
    solution_nodes_False = list()
    
    for ni in gvx.Nodes():
        n_id=ni.GetId()
        if n_id< num_nodes:
#             print('Node %d: %f' %(n_id, d_list[n_id]*gvx.GetNodeValue(n_id,'x'))) 
            solution_nodes_False.append(float(d_list[n_id]*gvx.GetNodeValue(n_id,'x')))
    
#     print(solution_nodes_False)

    solution_False = (sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))

#     print(sorted(solution_nodes_False)[-1], solution_nodes_False.index(sorted(solution_nodes_False)[-1]))
                
    
    return runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False



In [ ]:
node_lst = []
edge_lst = []

runtime_lst_True = []
runtime_lst_False = []

loss_lst_True = []
loss_lst_False = []

solutions_True = []
solutions_False = []

for n in [20, 50, 100, 500, 1000]:
    
    StarGraph_n = StarGraph(int(n))
    runtime_True, loss_True, solution_True, runtime_False, loss_False, solution_False = StarGraph_Solve(StarGraph_n, int(n))
    
    node_lst.append(n)
    edge_lst.append(n-1)
    
    runtime_lst_True.append(runtime_True)
    loss_lst_True.append(loss_True)
    solutions_True.append(solution_True)
    
    runtime_lst_False.append(runtime_False)
    loss_lst_False.append(loss_False)
    solutions_False.append(solution_False)


In [ ]:
with open('./StarGraph/StarGraph_Result.csv', 'w') as f:  
    f.write('nodes, edges, runtime_True, runtime_False, loss_True, loss_False, solution_possibility_True, solution_node_True, solution_possibility_False, solution_node_False\n')
    for n in range(len(node_lst)):
        n = int(n)
        f.write(str(node_lst[n])+','+str(edge_lst[n])+','+str(runtime_lst_True[n])+','+str(runtime_lst_False[n])+','+str(loss_lst_True[n])+','+str(loss_lst_False[n])+','+str(solutions_True[n][0])+','+str(solutions_True[n][1])+','+str(solutions_False[n][0])+','+str(solutions_False[n][1])+'\n')
f.close()